<a href="https://colab.research.google.com/github/autheblanc/skills-introduction-to-github/blob/main/1201_prediction_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import h5py
with h5py.File('/content/1201_prediction_analysis.h5', 'r') as f:
    occupancy_matrix = f['track_occupancy'][:]
    tracks_matrix = f['tracks'][:]

print(occupancy_matrix.shape)
print(tracks_matrix.shape)

(293, 1)
(1, 2, 3, 293)


In [2]:
with h5py.File( '/content/1201_prediction_analysis.h5', 'r') as f:
    dset_names = list(f.keys())
    locations = f["tracks"][:].T
    node_names = [n.decode() for n in f["node_names"][:]]

print("===filename===")
print('/content/1201_prediction_analysis.h5')
print()

print("===HDF5 datasets===")
print(dset_names)
print()

print("===locations data shape===")
print(locations.shape)
print()

print("===nodes===")
for i, name in enumerate(node_names):
    print(f"{i}: {name}")
print()

===filename===
/content/1201_prediction_analysis.h5

===HDF5 datasets===
['edge_inds', 'edge_names', 'instance_scores', 'labels_path', 'node_names', 'point_scores', 'provenance', 'track_names', 'track_occupancy', 'tracking_scores', 'tracks', 'video_ind', 'video_path']

===locations data shape===
(293, 3, 2, 1)

===nodes===
0: snout
1: eye
2: ear



In [3]:
frame_count, node_count, _, instance_count = locations.shape

print("frame count:", frame_count)
print("node count:", node_count)
print("instance count:", instance_count)

frame count: 293
node count: 3
instance count: 1


In [4]:
!pip uninstall -qqq -y opencv-python opencv-contrib-python
!pip install -qqq "sleap[pypi]>=1.3.3"


In [5]:
# Test video:
!wget https://storage.googleapis.com/sleap-data/reference/flies13/190719_090330_wt_18159206_rig1.2%4015000-17560.mp4

# Test video labels (from predictions/not necessary for inference benchmarking):
!wget https://storage.googleapis.com/sleap-data/reference/flies13/190719_090330_wt_18159206_rig1.2%4015000-17560.slp

# Bottom-up model:
# !wget https://storage.googleapis.com/sleap-data/reference/flies13/bu.210506_230852.multi_instance.n%3D1800.zip

# Top-down model (two-stage):
!wget https://storage.googleapis.com/sleap-data/reference/flies13/centroid.fast.210504_182918.centroid.n%3D1800.zip
!wget https://storage.googleapis.com/sleap-data/reference/flies13/td_fast.210505_012601.centered_instance.n%3D1800.zip

--2024-12-02 01:53:47--  https://storage.googleapis.com/sleap-data/reference/flies13/190719_090330_wt_18159206_rig1.2%4015000-17560.mp4
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.137.207, 142.250.101.207, 142.251.2.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.137.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85343812 (81M) [video/mp4]
Saving to: ‘190719_090330_wt_18159206_rig1.2@15000-17560.mp4.1’

190719_090330_wt_18 100%[===================>]  81.39M  46.0MB/s    in 1.8s    

2024-12-02 01:53:49 (46.0 MB/s) - ‘190719_090330_wt_18159206_rig1.2@15000-17560.mp4.1’ saved [85343812/85343812]

--2024-12-02 01:53:49--  https://storage.googleapis.com/sleap-data/reference/flies13/190719_090330_wt_18159206_rig1.2%4015000-17560.slp
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.137.207, 142.250.101.207, 142.251.2.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74

In [6]:
!ls -lah

total 295M
drwxr-xr-x 1 root root 4.0K Dec  2 01:53  .
drwxr-xr-x 1 root root 4.0K Dec  2 01:22  ..
-rw-r--r-- 1 root root  26K Dec  2 01:24  1201_prediction_analysis.h5
-rw-r--r-- 1 root root  82M May 20  2021  190719_090330_wt_18159206_rig1.2@15000-17560.mp4
-rw-r--r-- 1 root root  82M May 20  2021  190719_090330_wt_18159206_rig1.2@15000-17560.mp4.1
-rw-r--r-- 1 root root 1.6M May 20  2021  190719_090330_wt_18159206_rig1.2@15000-17560.slp
-rw-r--r-- 1 root root 1.6M May 20  2021  190719_090330_wt_18159206_rig1.2@15000-17560.slp.1
-rw-r--r-- 1 root root  59M Dec  2 01:39  241125_012328.single_instance.zip
-rw-r--r-- 1 root root 6.1M May 20  2021 'centroid.fast.210504_182918.centroid.n=1800.zip'
-rw-r--r-- 1 root root 6.1M May 20  2021 'centroid.fast.210504_182918.centroid.n=1800.zip.1'
drwxr-xr-x 4 root root 4.0K Nov 25 19:13  .config
drwxr-xr-x 1 root root 4.0K Nov 25 19:13  sample_data
-rw-r--r-- 1 root root  30M May 20  2021 'td_fast.210505_012601.centered_instance.n=1800.zip'
-rw-

In [9]:
import sleap

# This prevents TensorFlow from allocating all the GPU memory, which leads to issues on
# some GPUs/platforms:
sleap.disable_preallocation()

# This would hide GPUs from the TensorFlow altogether:
# sleap.use_cpu_only()

# Print some info:
sleap.versions()
sleap.system_summary()c

SLEAP: 1.3.4
TensorFlow: 2.8.4
Numpy: 1.22.4
Python: 3.10.12
OS: Linux-6.1.85+-x86_64-with-glibc2.35
GPUs: None detected.


In [10]:
# Videos can be represented agnostic to the backend format
video = sleap.load_video("/content/EC01_results_2024-11-08T10_20_2_Trial_23.avi")

# sleap.Video objects have a numpy-like interface:
print(video.shape)

# And we can load images in the video using array indexing:
imgs = video[:4]
print(imgs.shape, imgs.dtype)

(503, 480, 640, 3)
(4, 480, 640, 3) uint8


In [11]:
predictor = sleap.load_model("/content/241125_012328.single_instance.zip")

/usr/local/lib/python3.10/dist-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


In [12]:
labels = predictor.predict(video)
labels

Output()

Labels(labeled_frames=503, videos=1, skeletons=1, tracks=0)

In [13]:
labels.videos

[Video(backend=MediaVideo(filename='/content/EC01_results_2024-11-08T10_20_2_Trial_23.avi', grayscale=False, bgr=True, dataset='', input_format='channels_last'))]

In [14]:
labels.skeletons

[Skeleton(name='Skeleton-3', description='None', nodes=['snout', 'eye', 'ear'], edges=[], symmetries=[])]

In [21]:
labeled_frame = labels
labeled_frame

Labels(labeled_frames=503, videos=1, skeletons=1, tracks=0)

In [27]:
labeled_frame.instances

<bound method Labels.instances of Labels(labeled_frames=503, videos=1, skeletons=1, tracks=0)>

In [31]:
instance = labeled_frame  # shortcut for labeled_frame.instances[0]
instance

Labels(labeled_frames=503, videos=1, skeletons=1, tracks=0)

In [33]:
pts = instance.numpy()
print(pts)

[[[[620.3184  267.8072 ]
   [600.14667 248.22794]
   [568.1183  239.43674]]]


 [[[620.3187  267.79874]
   [600.1289  248.23192]
   [568.0957  239.46967]]]


 [[[620.3054  267.83078]
   [600.1061  248.2154 ]
   [568.0035  239.3862 ]]]


 ...


 [[[623.9064  267.9637 ]
   [603.66077 251.97357]
   [567.9694  243.95343]]]


 [[[623.89777 267.9253 ]
   [603.63385 251.92723]
   [567.8862  243.95735]]]


 [[[623.84393 267.9788 ]
   [603.6308  251.96356]
   [567.92523 243.97899]]]]
